In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy
from langdetect import detect
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


engine = create_engine("sqlite:///movies_dataset/movies.sqlite", connect_args={'check_same_thread': False})

In [2]:
#read in data
df = pd.read_csv("movies_dataset/movies_dirty.csv")

In [3]:
#Filter data from 2010 - 2016 (AO3 start date and end date of movie data
df["year"] = df["year"].loc[df["year"] >= 2010]

In [4]:
# drop NA
df = df.dropna()
#df

In [5]:
#set column types
df["name"] = df["name"].astype(object)
df["year"] = df["year"].astype(int)
df.dtypes

name         object
rating       object
genre        object
year          int32
released     object
score       float64
votes       float64
director     object
writer       object
star         object
country      object
budget      float64
gross       float64
company      object
runtime     float64
dtype: object

In [6]:
# Create and format movie_tag column to match data from AO3
df["movie_tag"] = df["name"] + " (" + df["year"].astype("string") + ")"
#df

In [7]:
# Save clean table
df.to_csv("movies_dataset/movie_clean.csv")

#Imports for scraping datta
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
# Scrape site and Parse the HTML
url = 'https://archiveofourown.org/media/Movies/fandoms'
browser.visit(url)
html = browser.html
html_soup = soup(html, 'html.parser')
# Scrape the number of fics written
title = []
tags_group = html_soup.find('ol', class_='alphabet fandom index group')
tags = tags_group.find_all("a", class_="tag")
amount = tags_group.find_all("li")
#print(tags_group)
for tag in tags:
    print(tag.text)
    title.append(tag.text)
    
    
# AO3 data list
list_ = []
for i in amount:
    print(i.text)
    list_.append(i.text)
    
    
# Scrape AO3
lists = []
tags = html_soup.find_all('ul', class_= "tags index group")
for tag in tags:
    print(tag.text)
    list_.append(tag.text)

browser.quit()

# Creating DataFrame for A03 
tag_df = pd.DataFrame(list_, columns=["movie_tag"])

# Save data
tag_df.to_csv("movies_dataset/ao3_dirty.csv")

In [8]:
#read in data
tag_df = pd.read_csv("movies_dataset/ao3_dirty.csv", index_col=0)
tag_df

,movie_tag
0,\n\n 0\n \n↑\n\n\n\n\n00 Sch...
1,\n00 Schneider - Jagd auf Nihil Baxter (1994)\...
2,\n\n 1\n \n↑\n\n\n\n\n1 Per ...
3,\n1 Per Cent | Outlaws (Australia Movie 2017)\...
4,\n10 Cloverfield Lane (2016)\n ...
...,...
10300,\n\nV for Vendetta - All Media Types\n ...
10301,\n\nW pustyni i w puszczy | In Desert and Wild...
10302,\n\nX (Movie 2022)\n (9)\n ...
10303,\n\nY tu mamá también (2001)\n ...


In [9]:
#clean data
tag_df["movie_tag"] = tag_df["movie_tag"].str.replace("\n", "")
tag_df = tag_df["movie_tag"].str.rsplit(n=1, expand=True)
tag_df.rename(columns={0:'movie_tag', 1:'fan_fic'}, inplace=True)
tag_df["fan_fic"] = tag_df["fan_fic"].str.strip("("")")
tag_df["fan_fic"] = tag_df["fan_fic"].astype(int)
tag_df.head()

,movie_tag,fan_fic
0,0 ↑00 Schneider - Jagd auf ...,1
1,00 Schneider - Jagd auf Nihil Baxter (1994),1
2,1 ↑1 Per Cent | Outlaws (Au...,2
3,1 Per Cent | Outlaws (Australia Movie 2017),2
4,10 Cloverfield Lane (2016),13


In [10]:
# Filter english only titles
def d_tect(x):
    try:
        ans=detect(x)
    except:
        ans='en'
    return ans

tag_df["lang"] = tag_df["movie_tag"].apply(d_tect)
tag_df = tag_df[tag_df["lang"]=="en"]

In [11]:
#reset index and drop lang column
tag_df.drop("lang", axis=1).reset_index(drop=True)

,movie_tag,fan_fic
0,1 ↑1 Per Cent | Outlaws (Au...,2
1,1 Per Cent | Outlaws (Australia Movie 2017),2
2,10 Cloverfield Lane (2016),13
3,10 Things I Hate About You (1999),136
4,The 10 Year Plan (2014),5
...,...,...
6207,V for Vendetta - All Media Types ...,12
6208,W pustyni i w puszczy | In Desert and Wilderne...,3
6209,X (Movie 2022) (9) X...,93
6210,Y tu mamá también (2001) (27) ...,1


In [12]:
# save to csv
tag_df.to_csv("movies_dataset/tag_clean.csv")

In [13]:
# Send to sqlite

#tag_df.to_sql('tag', con=engine)
#df.to_sql('movie', con=engine)

In [17]:
merged_inner = pd.merge(left=df, right=tag_df, left_on='movie_tag', right_on='movie_tag')
merged_inner.drop(["name", "rating", "year", "released", "director", "writer", "star", "country", "budget", "gross", "company", "runtime", "lang"], axis=1)
merged_inner.to_csv("movies_dataset/clean.csv")